# Seasonal consistency checks

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
from itertools import combinations
from itertools import combinations_with_replacement as cwr
import ipywidgets as widgets

In [2]:
data_dir = "../outputs/data/"
seasons = ["s17", "s18", "s19"]
arrays = ["pa4_f150", "pa4_f220", "pa5_f090", "pa5_f150", "pa6_f090", "pa6_f150"]
cross_arrays = ["%sx%s" % (m[0], m[1]) for m in cwr(arrays, 2)]
spectra = ["TT", "TE", "ET", "EE"]

In [3]:
# Spectra/Residual widget button
show = widgets.RadioButtons(
    description="Show",
    value="spectra",
    options=["spectra", "residuals"])

# Residual unit widget button
unit_residuals = widgets.RadioButtons(
    description="Unit",
    value="raw",
    options=["raw", "sigmas"])

# Mode widget
spectrum = widgets.Dropdown(
    value="TT",
    options=spectra)

# Cross array widget
cross = widgets.Dropdown(
    value=cross_arrays[0],
    options=cross_arrays)

header = widgets.HBox([show, unit_residuals, spectrum, cross])

import plotly.graph_objects as go
layout = dict(height=600, template="plotly_white", xaxis_title="$\ell$", 
              legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),)

In [4]:
fig = go.FigureWidget(layout=layout)
for s in seasons:
    fig.add_scatter(name=s, mode = "markers")

colors = ["red", "green", "blue"]
spectra_file = os.path.join(data_dir, "spectra.pkl")
residual_file = os.path.join(data_dir, "residual.pkl")
spec_dict = pickle.load(open(spectra_file, "rb"))
res_dict = pickle.load(open(residual_file, "rb"))

def update(_):
    spec = spectrum.value
    combin = cross.value
    pa1, pa2 = combin.split('x')
    
    fig.layout.title = "{} - {}".format(spec, combin)
    fig.layout.xaxis.title = r"$\ell$"
    
    if show.value == "spectra":
        fig.layout.yaxis.title = r"$D_\ell^{\mathrm{%s}}\;[\mu\mathrm{K}^2]$" % spec
        fig.update_yaxes(type="log" if spec == "TT" else "linear", autorange=True)
    if show.value == "residuals":
        if unit_residuals.value == "raw":
            fig.layout.yaxis.title = r"$D_\ell^{\mathrm{%s}, 18} - D_\ell^{\mathrm{%s}, 19}\;[\mu\mathrm{K}^2]$" % (
                    spec, spec)
            
        elif unit_residuals.value == "sigmas":
            fig.layout.yaxis.title = r"$D_\ell^{\mathrm{%s}, 18} - D_\ell^{\mathrm{%s}, 19}\;[\sigma]$" % (spec, spec)
        fig.update_yaxes(type="linear", autorange=True)
    
    if show.value == "spectra":
        for i, season in enumerate(seasons):
            with fig.batch_update():
                current = fig.data[i]
                current.mode = "markers"
                current.x = spec_dict[spec, season, pa1, pa2][0]
                current.y = spec_dict[spec, season, pa1, pa2][1]
                current.error_y.array = spec_dict[spec, season, pa1, pa2][2]
                current.marker.color = colors[i]
                current.name = season
        
    elif show.value == "residuals":
        for i, (season1, season2) in enumerate(combinations(seasons, 2)):
    
            if unit_residuals.value == "raw":
                y = res_dict[spec, season1, season2, pa1, pa2][1]

            elif unit_residuals.value == "sigmas":
                y = res_dict[spec, season1, season2, pa1, pa2][1] / (
                    res_dict[spec, season1, season2, pa1, pa2][2])

                
            with fig.batch_update():
                current = fig.data[i]
                if unit_residuals.value == "sigmas":
                    current.mode = "lines+markers"
                else: 
                    current.mode = "markers"
                current.x = res_dict[spec, season1, season2, pa1, pa2][0]
                current.y = y
                current.line.color = colors[i]
                current.marker.color = colors[i]
                if unit_residuals.value == "sigmas":
                    current.error_y.array = None
                else:
                    current.error_y.array = res_dict[spec, season1, season2, pa1, pa2][2]
                current.name = "%s - %s" % (season1, season2)
                
update(None)

show.observe(update, names="value")
unit_residuals.observe(update, names ="value")
spectrum.observe(update, names="value")
cross.observe(update, names="value")